<h1><center>A PHYLOGENETIC ANALYSIS OF POKEMON</center></h1>

![alt text](image1.jpeg "Title")

# INTRODUCTION
The purpose of this compiler is to build a 'genetic' database of Pokemon (also known as Poket Monsters, ポケットモンスター, or Poketto Monsutā) based on their explicit and unique 'move sets', 'body shapes', 'egg groups', and 'types'.  For context, the data being compiled is from a fantasy game developed by Nintendo in the mid 1990's that has grown in massive popularity over the past few decades.  The game's titular characters are a type of organism that share a single ancestor and have mutated and evolved into various forms based on environment and stress.  The gameplay pits these organisms against each other in paper-rock-scissors style combat around strengths and weaknesses; there is a competative scene around this gameplay in the real world.  Looking into the nuances of how each Pokemon stacks up against its relatives becomes as gritty as number crunching based on statistics AND move-set managemant based on 'evolutionary form'.

Website resources exist for the purpose of helping those interested in this level of competetion.  From official databases to meticulously updated fan-sites and wikis, there is no shortage of data, but they are all in disparate forms.  Amongst all this raw data, there existed a very interesting pattern, that seemed to mimic genetic sequences that I have worked with before. Repeated sequences in moves, shared breeding groups, and fundamental typing became the basis of that pattern.  Through application of 'translation' from raw data to processable genetic sequences development of a phylogenetic tree with about 90% accuracy was developed based on the first 250 organisms (image above).  

This is reworking (v2) of that original web-scrapper and compiler utilizing Pandas, NumPy, Beautiful Soup, and UrlLib.  The goal of this notebook is to sheperd the process of developing this project into a GitHub resource.  The final accomplishment of a working program should compile the raw data for a list of Pokemon into a CSV file (FinalDatabaseYYYY.MM.DD-HH.MM.SS.csv) for downstream phylogenetics translation.  Later additions to this will include the translation of the raw data to genetic information, analysis of molecular variance, phylogenetic processing, and tree imaging.

### Data Source Description
The data sources, from Git Repositories to the myriad of websites, all contain disparate data about the organisms that needs to be compiled together: Serebii has move sets, egg groups, form variations; while the Git Repositories contain base stats, alternate forms, as well as organism types.  Furthermore, the other resources will be the source of Body Types and Location data to further aid in possible phylogeographic analysis.  There are going to be mostly string data coming from these resources, this is optimal as it will provide the best format for 'translation'.  The string data will be appended to the current dataframe as extra columns.

#### GitHub Repository: 
This is a fork of the CSV curated by "simsketch". I used it as a basline, added in the most recent releases and corrected a few errors in some later entries. I leave the repository as a fork and pull it here as a Pandas DF. This is the source for Abilities, Types, Stats, and Generation Debut parameters.

#### Bulbapedia
Bulbapedia is the source for the Body Type parameter. This comes from an image in all Pokemon entries.

#### Serebii
Currently only Move-sets (Level, Form, TM/HM/TR, Egg Moves, Move Tutor Attackes, Max Moves, and Z Moves) are extracted from Serebii. This comes from a table with words and images in all Pokemon entries.

#### PokemonDB
PokemonDB is the source for the Egg Group, Gender Ratios, Steps-to-Hatch, and Locations parameters. This comes from a series of tables, and only Egg Group is currently used.

##### Known Issues:
Currently uses index of 1000+ entries instead of pokedex number to update search

### Results & Discussion
The final results of this project currently is an updated CSV file (FinalDatabaseYYYY.MM.DD-HH.MM.SS.csv) that contains updated data from multiple resources (Bulbapeida, PokemonDB, Serebii & Git Repository now).  This file will then be used as a basis for 'translation' into genetic sequences that can then have their variance calculated through AMOVA (Analysis of Molecular Variance) which will then provide a means to produce another phylogenetic tree.

# CODE TABLE OF CONTENTS:
#### Section 1) Import Libraries
#### Section 2) General Utilities
#### Section 3) Specific Utilities
#### Section 4) Dataframe Compilers
#### Section 5) Group Search - Database Builder
#### Section 6) Single Search - Research Display


### 1) Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import sys
from pathlib import Path
import time
from datetime import datetime
from Bio import Phylo
from Bio.Phylo.TreeConstruction import *
from Bio import AlignIO
from io import StringIO
from more_itertools import unique_everseen
import os
import glob

# disable chained assignments
pd.options.mode.chained_assignment = None 

# Calls in necessary library contianing needed lists and gene translator
%run ./Libraries/GeneProcessors.ipynb
%run ./Libraries/CompilerLists.ipynb

# Gather Date/Time for Saving final CSV
DATE_TIME = datetime.today().strftime('%Y.%m.%d-%H.%M.%S')

LOADING DICTIONARY
MOVES DATABSE FILE EXISTS: 850 ENTRIES - Loaded
SAVING UPDATED DATAFRAME: EG_DICT.csv


###  2) General Utilities

In [32]:
###
# General Utilities made-for use in any program
###
def get_soup(URL):  
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    print(URL + '\n')
    SOUP = BeautifulSoup(requests.get(URL, verify=False).content, "html.parser")
    return SOUP

def progress(count, total, status=''):
    bar_len = 25
    filled_len = int(round(bar_len * count / float(total)))
    percents = round(100.0 * count / float(total), 1)
    bar = '█' * filled_len + '-' * (bar_len - filled_len)
    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()
    
def list_duplicate_removal(input_list):
    output_list = []
    for item in input_list:
        if item not in output_list:
            output_list.append(item)
    return output_list

def remove_blank_first_df_col(inputCSVPath):
    df = pd.read_csv(inputCSVPath)
    # If you know the name of the column skip this
    first_column = df.columns[0]
    if 'Unnamed: 0' in first_column:
        # Delete first
        df = df.drop([first_column], axis=1)
        df.to_csv(inputCSVPath, index=False)
    else:
        pass
    
def remove_csv_duplicates(tempCSVPath,finalCSVPath):
    with open(tempCSVPath,'r') as f:
        with open(finalCSVPath,'w') as out_file:
            out_file.writelines(unique_everseen(f))
            
def csv_cleanup(excludedFile):
    # Get a list of all the file paths that ends with .csv from in specified directory
    fileList = glob.glob('./*.csv')
    # Iterate over the list of filepaths & remove each file, preserving the excludedFile.
    for filePath in fileList:
        if filePath == f'.\\{excludedFile}':
            print(f'Preserving: {filePath}')
            pass
        else:
            try:
                print(f'Removing: {filePath}')
                os.remove(filePath)
            except:
                print("Error while deleting file : ", filePath)

### 3) Specific Utilities

In [38]:
###
# Main Functions for List, Name, and Indecies Management
###
def get_full_list():
    DOWNLOAD_CSV = Path('./BASE_DATABASE.csv')
    if DOWNLOAD_CSV.exists():
        fullListDF = pd.read_csv(DOWNLOAD_CSV, index_col=0)
        ENTRY_COUNT = len(fullListDF.index)
        print('DATABSE FILE EXISTS: ' + str(ENTRY_COUNT-1) + ' ENTRIES - Loaded')
    else:
        print('DATABSE FILE DOES NOT EXIST - Downloading')
        fullListDF = pd.read_csv(GIT_CSV, on_bad_lines='warn')
        fullListDF.fillna('-', inplace=True)
        fullListDF = fullListDF.drop(columns=['COLOR','MEGA_EVOLUTION','LEGENDARY', 'HEIGHT', 'WEIGHT', 'TOTAL'])
        fullListDF['REGION'] = ''
        fullListDF['BODY_TYPE'] = ''
        fullListDF['EGG_GROUP1'] = ''
        fullListDF['EGG_GROUP2'] = ''
        fullListDF['STEPS_TO_HATCH'] = ''
        fullListDF['LOCATIONS'] = ''
        fullListDF['MOVES'] = ''
        fullListDF['GENDER_RATIOS(M:F)'] = ''
        fullListDF['GENERATION'] = fullListDF['GENERATION'].astype(str)
        fullListDF['BODY_TYPE'] = fullListDF['BODY_TYPE'].astype(str)
        fullListDF['EGG_GROUP1'] = fullListDF['EGG_GROUP1'].astype(str)
        fullListDF['EGG_GROUP2'] = fullListDF['EGG_GROUP2'].astype(str)
        fullListDF['GENDER_RATIOS(M:F)'] = fullListDF['GENDER_RATIOS(M:F)'].astype(str)
        fullListDF['LOCATIONS'] = fullListDF['LOCATIONS'].astype(str)
        fullListDF = fullListDF[FINAL_ORDER_LIST]
        for index, row in fullListDF.iterrows():
            generationDebut = GENERATIONS_LIST[int(row['GENERATION'])]
            if generationDebut == 'Galarian/Hisuian':
                if row['NAME'] in GALARIAN_FORM_LIST:
                    generationDebut = 'Galarian'
                elif row['NAME'] in GALARIAN_DEBUT_LIST:
                    generationDebut = 'Galarian'
                elif row['NAME'] in HISUIAN_FORM_LIST:
                    generationDebut = 'Hisuian'
                elif row['NAME'] in HISUIAN_DEBUT_LIST:
                    generationDebut = 'Hisuian'
                else:
                    generationDebut = '*NEEDS WORK*'
            fullListDF.at[index,'REGION'] = generationDebut
        ENTRY_COUNT = len(fullListDF.index)
        fullListDF.to_csv(DOWNLOAD_CSV)  
        print('DATABSE FILE DOWNLOADED: '+ str(ENTRY_COUNT-1) +' ENTRIES - Saved')
    return fullListDF

# Pulls all entries from the Base Database File for updating.
def pull_entries(fullListDF,INPUT1=None, INPUT2=None):
    try:
        if INPUT1.isalpha() == True:
            print('Input 1 was Pokemon Name')
            NAME = INPUT1
        else:
            print('Input 1 was Pokedex Number')
            DEXNUMB = INPUT1
    except AttributeError:
        print('Input 1 was Pokedex Number')
        DEXNUMB = INPUT1
    if INPUT2 == None:
        pass
    else:
        if INPUT2.isalpha() == True:
            print('Input 2 was Pokemon Name')
            NAME = INPUT2
        else:
            print('Input 2 was Pokedex Number')
            DEXNUMB = INPUT2
    if 'DEXNUMB' in locals():
        currentEntriesDF = fullListDF[fullListDF['NUMBER'] == DEXNUMB]
        NAME = currentEntriesDF.iloc[0][5]
    elif 'NAME' in locals():
        currentEntriesDF = fullListDF[fullListDF['NAME'] == NAME]
        DEXNUMB = str(currentEntriesDF.iloc[0][0])   
    display(currentEntriesDF)
    currentIndicies = list(currentEntriesDF.index.values)
    print("CURRENT INDICIES")
    print(currentIndicies)
    refName = name_checker(NAME, 3)
    print('Returning Entries for: #'+ str(DEXNUMB) +' - '+ refName)
    return currentEntriesDF, currentIndicies, refName, NAME, DEXNUMB

# Checks to see if the Name has any regular expressions or irregulatires to it.
def name_checker(NAME, REFERENCE):
    if NAME in NAME_IRREGULARITIES_DICT:
        UPDATED_NAME = NAME_IRREGULARITIES_DICT[NAME][REFERENCE]
    else:
        UPDATED_NAME = NAME
    return UPDATED_NAME

# Checks to see if there are alternate forms based on the Name provided.
def alt_form_updater(NAME, currentIndicies, currentEntries):
    for index in currentIndicies:
        if NAME in ALT_FORM_DICT:
            altFormCheck = currentEntries['CODE'][index]
            formName = ALT_FORM_DICT[NAME][altFormCheck-1] + ' ' +NAME
            currentEntries.loc[index, 'NAME'] = formName
            
# Checks to see if there are alternate body types based on the Name provided.            
def alt_body_updater(NAME, currentIndicies, currentEntries):
    for index in currentIndicies:
        tempName = currentEntries['NAME'][index]
        if tempName in ALT_BODY_DICT:
            formBody = ALT_BODY_DICT[tempName]
            currentEntries.loc[index, 'BODY_TYPE'] = formBody

# Updates the Base Database based on the entries requested and makes a FinalDatabase from it then cleans up unwanted CSVs.             
def main_updater(fullListDF):
    UPDATED_DF = pd.DataFrame()
    pullStartRange = int(input("What Entry Number would you like to start at?: "))
    pullEndRange = int(input("What Entry Number would you like to go up to?: "))
    pullCount = pullStartRange
    while pullCount < pullEndRange:
        searchNumb = fullListDF['NUMBER'][pullCount]
        currentEntries, currentIndicies, refName, NAME, DEXNUMB = pull_entries(fullListDF,searchNumb)
        BPimageDictionary = bulbapedia_images(NAME)
        PDBtablesDictionary, PDBtableCount = pokemondb_tables(NAME)
        SBtablesDictionary = serebii_tables(NAME, DEXNUMB)
        BPparsed = bulbapedia_parser(BPimageDictionary, currentEntries, currentIndicies, NAME)
        PDBparsed = pokemondb_parser(PDBtablesDictionary, PDBtableCount, currentEntries, currentIndicies, refName, NAME)
        SBparsed = serebii_parser(SBtablesDictionary, currentEntries, currentIndicies, NAME)
        alt_form_updater(NAME, currentIndicies, currentEntries)
        alt_body_updater(NAME, currentIndicies, currentEntries)
        UPDATED_DF = UPDATED_DF.append(currentEntries, ignore_index = True)
        pullCount = pullCount+1
    display(UPDATED_DF)
    print(type(UPDATED_DF))
    UPDATED_DF = pd.DataFrame(UPDATED_DF)
    tempCSVPath = f'temp{DATE_TIME}.csv'
    UPDATED_DF.to_csv(tempCSVPath)
    finalCSVPath = f'FinalDatabase{DATE_TIME}.csv'
    remove_blank_first_df_col(tempCSVPath)
    remove_csv_duplicates(tempCSVPath,finalCSVPath)
    print(f'Final CSV Saved as: {finalCSVPath}')
    csv_cleanup(finalCSVPath)
    return(finalCSVPath)


###
# Bulbapedia Image Grabber -> Body Type Interpreter & Updater
###
def bulbapedia_images(NAME):    
    URL_NAME = name_checker(NAME,0)  
    URL = 'https://bulbapedia.bulbagarden.net/wiki/' + URL_NAME + '_(Pok%C3%A9mon)'
    SOUP = get_soup(URL)
    BPimageDictionary = {}
    BPimages = SOUP.find_all('img')
    BPimageCount = len(BPimages)
    print('Acquiring ' + str(BPimageCount) + ' Images for ' + URL_NAME + ' from Bulbapedia')
    for i, image in enumerate(BPimages):
        if '-Body' in image['src']:
            BPimageDictionary[i] = image['src']    
        i = i+1
        progress(i,BPimageCount,'Parsing')        
    print('\n')
    return BPimageDictionary

def bulbapedia_parser(BPimageDictionary, currentEntries, currentIndicies, refName):    
    if bool(BPimageDictionary) == False:
        bodyTemp = ['Unknown']
    else:        
        bulb_body_type(BPimageDictionary, currentEntries, currentIndicies)
    
def bulb_body_type(imageDictionary, currentEntries, currentIndicies):
    for entry in imageDictionary:
        bodyTemp = int(imageDictionary[entry].split('32px-Body')[1].split('.png')[0])
    bodyTemp = BODY_TYPE_LIST[bodyTemp-1]
    currentEntries['BODY_TYPE'] = currentEntries['BODY_TYPE'].astype(str)
    for index in currentIndicies:
        currentEntries.at[index, 'BODY_TYPE'] = bodyTemp
    print('Updated Body Type for all entries of ' + currentEntries['NAME'])
    
###
# PokemonDB Table Grabber -> Egg Group, Gender Ratios, Steps-to-Hatch, and Locations Interpreter & Updater
###
def pokemondb_tables(NAME):  
    URL_NAME = name_checker(NAME,1)
    URL = 'https://pokemondb.net/pokedex/' + URL_NAME
    SOUP = get_soup(URL)
    PDBtables = SOUP.find_all('table')
    PDBtablesDictionary = {}
    PDBtableCount = len(PDBtables)
    print('Acquiring ' + str(PDBtableCount) + ' Tables for ' + URL_NAME + ' from PokemonDB')
    for t,tables in enumerate(PDBtables):
        dfTemp = pd.read_html(str(PDBtables))[t]
        PDBtablesDictionary[t] = dfTemp
        t+=1
        progress(t,PDBtableCount,'Collected')
    print('\n')
    return PDBtablesDictionary, PDBtableCount

def pokemondb_parser(PDBtablesDictionary, PDBtableCount, currentEntries, currentIndicies, refName, NAME):  
    EggGroupTemp = PDBtablesDictionary[2][1][0]
    if '—' in EggGroupTemp:
        EggGroup1 = 'Unknown'
        EggGroup2 = 'Unknown'
    elif ', ' in EggGroupTemp:
        EggGroup1 = PDBtablesDictionary[2][1][0].split(', ')[0]
        EggGroup2 = PDBtablesDictionary[2][1][0].split(', ')[1]
    else:
        EggGroup1 = PDBtablesDictionary[2][1][0]
        EggGroup2 = '-'
    genderRatioRaw = PDBtablesDictionary[2][1][1]
    if genderRatioRaw == '—':
        genderRatiosTemp = 'Unknown'
    elif genderRatioRaw == 'Genderless':
        genderRatiosTemp = genderRatioRaw
    else:
        genderRatioM = genderRatioRaw.split(' male, ')[0]
        genderRatioF = genderRatioRaw.split( 'male, ')[1].replace(' female','')
        genderRatiosTemp = genderRatioM + ':' + genderRatioF

    stepsTemp = PDBtablesDictionary[2][1][2]
    if stepsTemp == '—':
        stepsTemp = 'Unknown'
    else:
        stepsTemp = stepsTemp.split('(')[1].split('steps)')[0]
        stepsLow = stepsTemp.split('–')[0]
        stepsHigh = stepsTemp.split('–')[1]
        stepsTemp = str(stepsLow) + '-' + str(stepsHigh)

    finalLocations = []
    print(NAME)
    if NAME in HISUIAN_DEBUT_LIST:
        locationsTemp = 'Unknown'
        finalLocations.append(locationsTemp)
    else:
        locationsTemp = PDBtablesDictionary[PDBtableCount-2] 
        locTemp = locationsTemp[1].values.tolist()
        locTemp = list_duplicate_removal(locTemp)
        removeList = ['Trade/migrate from another game','Not available in this game']
        currentEntries['LOCATIONS'] = currentEntries['LOCATIONS'].astype(str)
        for l,location in enumerate(locTemp):
            if location in removeList:
                pass
            elif 'Evolve ' in location:
                pass
            elif ',' in location:
                splitLoc = location.split(', ')
                for s, split in enumerate(splitLoc):
                    if split.isnumeric() == True:
                        newSplit = 'Route ' + str(split)
                    else:
                        newSplit = split
                    finalLocations.append(newSplit)
            else:
                finalLocations.append(location)
                
    for index in currentIndicies:
        currentEntries.loc[index, 'EGG_GROUP1'] = EggGroup1
        currentEntries.loc[index, 'EGG_GROUP2'] = EggGroup2
        currentEntries.loc[index, 'GENDER_RATIOS(M:F)'] = genderRatiosTemp
        currentEntries.loc[index, 'STEPS_TO_HATCH'] = stepsTemp
        try:
            currentEntries.at[index, 'LOCATIONS'] = finalLocations
        except ValueError:
            currentEntries.loc[index, 'LOCATIONS'] = finalLocations
    print('Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of ' + refName)
    
###
# Serebii Table Grabber -> Move Set Interpreter & Updater
###
def serebii_tables(NAME, DEXNUMB):
    URL_NAME = name_checker(NAME,2)  
    URL = 'https://www.serebii.net/pokedex-swsh/' + URL_NAME.lower()
    SOUP = get_soup(URL)
    SBtablesDictionary = {}
    SBtables = SOUP.find_all('table')
    SBtableCount = len(SBtables)
    if SBtableCount == 0:        
        print('Generation VIII Data Not Available, using Generation VII Data')
        if len(str(DEXNUMB)) == 1:
            URLDEXNUMB = '00' + str(DEXNUMB)
        elif len(str(DEXNUMB)) == 2:
            URLDEXNUMB = '0' + str(DEXNUMB)
        elif len(str(DEXNUMB)) >= 3:
            URLDEXNUMB = str(DEXNUMB)
        URL = 'https://www.serebii.net/pokedex-sm/' + URLDEXNUMB + '.shtml'
        SOUP = get_soup(URL)
        SBtables = SOUP.find_all('table')
        SBtableCount = len(SBtables)
    print('Acquiring ' + str(SBtableCount) + ' Tables for ' + URL_NAME + ' from Serebii')
    for t, table in enumerate(SBtables):
        try:
            dfTemp = pd.read_html(str(SBtables))[t]
        except IndexError:
            dfTemp = 'IndexError at table['+ str(t) + ']'
        SBtablesDictionary[t] = dfTemp
        t+=1
        progress(t,SBtableCount,'Collected')
    print('\n')
    return SBtablesDictionary

def SBtablesDictionary_to_list(searchTable, movesFull):
    diffTables = ['Egg Moves (Details)','Usable Z Moves','Usable Max Moves'] # eventually if searchTable[0][0] == NaN is a possible catch all
    if searchTable[0][0] in diffTables:
        movesTemp = searchTable[0]
    else:   
        movesTemp = searchTable[1]
    movesTemp = movesTemp.reset_index(drop=True)
    movesTemp.columns = movesTemp.iloc[0] 
    movesTemp = movesTemp[1:]
    movesTemp = movesTemp.values.tolist()
    movesTemp = list_duplicate_removal(movesTemp)
    movesFull.extend(movesTemp[1:])
    print(searchTable[0][0] + ' added to Full Moves List')
    return movesTemp

def serebii_parser(SBtablesDictionary, currentEntries, currentIndicies, refName):
    currentEntries['MOVES'] = currentEntries['MOVES'].astype(str)
    for index in currentIndicies:
        tempRegion = currentEntries['REGION'][index]
        movesFull = []
        for table in SBtablesDictionary:
            try:
                searchTable = SBtablesDictionary[table]
                if 'M/F' in refName:
                    if index < currentIndicies[len(currentIndicies)-1]:
                        if searchTable[0][0] == 'Level Up - Male':
                            maleMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                    else:
                        if searchTable[0][0] == 'Level Up - Female':
                            femaleMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                if 'Galarian' in tempRegion:
                    if searchTable[0][0] == 'Galarian Form Level Up':
                        galMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                elif 'Alolan' in tempRegion:
                    if searchTable[0][0] == 'Alola Form Level Up':
                            aloMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                else:
                    if searchTable[0][0] == 'Standard Level Up':
                        stdMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                    elif searchTable[0][0] == 'Generation VII Level Up':
                        gen7Moves = SBtablesDictionary_to_list(searchTable, movesFull)        
                if searchTable[0][0] == 'Sword & Shield Technical Machine Attacks':
                    swshtmMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                if searchTable[0][0] == 'Technical Machine Attacks':
                    tmMoves = SBtablesDictionary_to_list(searchTable, movesFull)     
                if searchTable[0][0] == 'Technical Record Attacks':
                    trMoves = SBtablesDictionary_to_list(searchTable, movesFull)          
                if searchTable[0][0] == 'BDSP Technical Machine Attacks':
                    bdsptmMoves = SBtablesDictionary_to_list(searchTable, movesFull)          
                if searchTable[0][0] == 'TM & HM Attacks':
                    tmhmMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                if searchTable[0][0] == 'Egg Moves (Details)':   
                    eggMoves =SBtablesDictionary_to_list(searchTable, movesFull)
                if searchTable[0][0] == "Ultra Sun/Ultra Moon Move Tutor Attacks":
                    usummtMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                if searchTable[0][0] == "Usable Max Moves":
                    maxMoves = SBtablesDictionary_to_list(searchTable, movesFull)
                if searchTable[0][0] == 'Usable Z Moves':
                    zMoves = SBtablesDictionary_to_list(searchTable, movesFull)
            except:
                pass
        print('FULL MOVE SET COMPILED')
        for m,move in enumerate(movesFull):
            if 'BDSP Only' in move:
                movesFull[m] = move.replace('BDSP Only','')
            elif 'USUM Only' in move:
                movesFull[m] = move.replace('USUM Only','')
        currentEntries.at[index, 'MOVES'] = movesFull
    print('Updated Full Move Set for all entries of ' + refName)

# SPACE FOR FUTURE PROCESSING
def pdb_egg_Groups():
    pass

def pdb_locations():
    pass

### Section 4) Dataframe Compilers

In [34]:
fullListDF = get_full_list()
display(movesRNADF)
display(eggGroupsRNADF)
display(bodyTypesRNADF)

DATABSE FILE DOES NOT EXIST - Downloading
DATABSE FILE DOWNLOADED: 1058 ENTRIES - Saved


,Name,RNA_Seq
0,Pound,UUAGAGCUGUUACUG
1,Karate Chop,CCUGAGCCACCUCCG
2,Double Slap,UUAGAGUUGUUACUA
3,Comet Punch,UUAGAGCUUUUACUA
4,Mega Punch,UUAGAGCUUCUUCUA
...,...,...
845,Bleakwind Storm,GGAAACGGAGGGGGC
846,Wildbolt Storm,AUUAACAUUAUCAUA
847,Sandsear Storm,GAAAACGAAGAAGAG
848,Lunar Blessing,AAAUGGAAA------


,Input,Nucleotide Sequence
0,Fae(Fairy),AUGUGUGCUUGUUGUGUUAUUAAUUAUCCUCAUUUAAGGUGUGAUA...
1,Plant&Ha-Plants(Grass),AUGUGUGCUGAACAAGUUGCUGCUUAUCCUCAUGCUAGGGCUGAUA...
2,Mineral,AUGUGUGCUGAACAAGUUAUUAAUUAUCCUCAUUUAGAAAAUGAUA...
3,Ryu(Dragon),AUGCAAGCUGAACAACAAAUUAAUUAUCCUCAAUUAAGGAAUGAUC...
4,Arthropods & Mullosks(Bug),AUGUGUGCUGAACAAGUUAUUAAUGUUCCUCAUGUUAGGAAUGAUA...
5,Human-Like,AUGUGUGCUUUAUUAGUUAUUUUAUAUCCUCAUUUAAGGAAUGAUA...
6,Monster,AUGCCUGCUGAACAACCUAUUAAUUAUCCUCAUUUAAGGAAUGAUA...
7,Ghost&Soft-Bodied(Amorphous),AUGUGUGCUGAACAAUAUAUUAAUUAUCCUCAUUUAAGGAAUGAUA...
8,Aquatic-Terrestrial(W1),AUGUGUGCUGAACAAGUUAUUAAUUAUCCUCAUUUAAGGAGUGAUA...
9,Fish-Cephelopod(W2),AUGAGUAGUGAAAGUAGUAUUAAUAGUAGUAGUAGUAGGAGUGAUA...


,Input,Nucleotide Sequence
0,Bipedal,AGUAUUGCAUUU
1,Quadraped,AGCAUUGCGUUU
2,Bipedal&Tailed,AGUAUCGCAUUU
3,Head&Arms,AGUAUUGCAUUC
4,Head&Legs,AGUAUUGCGUUU
5,Head&Base,AGUAUUGCGUUC
6,OnlyHead,AGUAUAGCGUUC
7,Finned,AGUAUCGCUUUU
8,Serpentine,AGUAUCGCGUUC
9,SingleWings,AGUAUUGCCUUC


### Section 5) Group Search - Database Builder

In [42]:
# 192 = Mew
# 300 = Celebi
# 463 = Deoxyis
#ERROR AT 656-569 Darmanitan
# 593 = Arceus
# 770 = Genesect
# 855 = Volcanion
# 954 = Melmetal
# 1058 = Enamorus

finalCSVPath = main_updater(fullListDF)

What Entry Number would you like to start at?: 660
What Entry Number would you like to go up to?: 1059
Input 1 was Pokedex Number


,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
660,556,1,5561,5,Unovan,Maractus,,,,Grass,...,,,75,86,67,106,67,60,,


CURRENT INDICIES
[660]
Returning Entries for: #556 - Maractus
https://bulbapedia.bulbagarden.net/wiki/Maractus_(Pok%C3%A9mon)

Acquiring 116 Images for Maractus from Bulbapedia
[█████████████████████████] 100.0% ...Parsing

https://pokemondb.net/pokedex/Maractus

Acquiring 24 Tables for Maractus from PokemonDB
[█████████████████████████] 100.0% ...Collected

https://www.serebii.net/pokedex-swsh/maractus

Acquiring 39 Tables for Maractus from Serebii
[█████████████████████████] 100.0% ...Collected

660    \nUpdated Body Type for all entries of Maractus
Name: NAME, dtype: object
Maractus
Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of Maractus
Standard Level Up added to Full Moves List
Technical Machine Attacks added to Full Moves List
Technical Record Attacks added to Full Moves List
Egg Moves (Details) added to Full Moves List
Usable Max Moves added to Full Moves List
FULL MOVE SET COMPILED
Updated Full Move Set for all entries of Maractus
Input 1 was 

,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
661,557,1,5571,5,Unovan,Dwebble,,,,Bug,...,,,50,65,85,35,35,55,,


CURRENT INDICIES
[661]
Returning Entries for: #557 - Dwebble
https://bulbapedia.bulbagarden.net/wiki/Dwebble_(Pok%C3%A9mon)

Acquiring 126 Images for Dwebble from Bulbapedia
[█████████████████████████] 100.0% ...Parsing

https://pokemondb.net/pokedex/Dwebble

Acquiring 20 Tables for Dwebble from PokemonDB
[█████████████████████████] 100.0% ...Collected

https://www.serebii.net/pokedex-swsh/dwebble

Acquiring 39 Tables for Dwebble from Serebii
[█████████████████████████] 100.0% ...Collected

661    \nUpdated Body Type for all entries of Dwebble
Name: NAME, dtype: object
Dwebble
Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of Dwebble
Standard Level Up added to Full Moves List
Technical Machine Attacks added to Full Moves List
Technical Record Attacks added to Full Moves List
Egg Moves (Details) added to Full Moves List
Usable Max Moves added to Full Moves List
FULL MOVE SET COMPILED
Updated Full Move Set for all entries of Dwebble
Input 1 was Pokedex Num

...

CURRENT INDICIES
[1057, 1058]
Returning Entries for: #905 - Enamorus
https://bulbapedia.bulbagarden.net/wiki/Enamorus_(Pok%C3%A9mon)

Acquiring 36 Images for Enamorus from Bulbapedia
[█████████████████████████] 100.0% ...Parsing

https://pokemondb.net/pokedex/Enamorus

Acquiring 18 Tables for Enamorus from PokemonDB
[█████████████████████████] 100.0% ...Collected

https://www.serebii.net/pokedex-swsh/enamorus

Acquiring 38 Tables for Enamorus from Serebii
[█████████████████████████] 100.0% ...Collected

1057    \nUpdated Body Type for all entries of Enamorus
1058    \nUpdated Body Type for all entries of Enamorus
Name: NAME, dtype: object
Enamorus
Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of Enamorus
Standard Level Up added to Full Moves List
FULL MOVE SET COMPILED
Standard Level Up added to Full Moves List
FULL MOVE SET COMPILED
Updated Full Move Set for all entries of Enamorus
Input 1 was Pokedex Number


,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
1057,905,1,9051,8,Hisuian,Enamorus,,,,Fairy,...,,,74,115,70,135,80,106,,
1058,905,2,9052,8,Hisuian,Enamorus,,,,Fairy,...,,,74,115,110,135,100,46,,


CURRENT INDICIES
[1057, 1058]
Returning Entries for: #905 - Enamorus
https://bulbapedia.bulbagarden.net/wiki/Enamorus_(Pok%C3%A9mon)

Acquiring 36 Images for Enamorus from Bulbapedia
[█████████████████████████] 100.0% ...Parsing

https://pokemondb.net/pokedex/Enamorus

Acquiring 18 Tables for Enamorus from PokemonDB
[█████████████████████████] 100.0% ...Collected

https://www.serebii.net/pokedex-swsh/enamorus

Acquiring 38 Tables for Enamorus from Serebii
[█████████████████████████] 100.0% ...Collected

1057    \nUpdated Body Type for all entries of Enamorus
1058    \nUpdated Body Type for all entries of Enamorus
Name: NAME, dtype: object
Enamorus
Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of Enamorus
Standard Level Up added to Full Moves List
FULL MOVE SET COMPILED
Standard Level Up added to Full Moves List
FULL MOVE SET COMPILED
Updated Full Move Set for all entries of Enamorus


,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
0,556,1,5561,5,Unovan,Maractus,Head & Base,Grass,-,Grass,...,"[Spiky Shield, Peck, Absorb, After You, Ingrai...",50%:50%,75,86,67,106,67,60,"4,884-5,140","[Desert Resort, Mirage Spots, Route 6, Bridge ..."
1,557,1,5571,5,Unovan,Dwebble,Insectoid,Bug,Mineral,Bug,...,"[Fury Cutter, Sand Attack, Withdraw, Smack Dow...",50%:50%,50,65,85,35,35,55,"4,884-5,140","[Route 18, Desert Resort, Desert Resort, Route..."
2,558,1,5581,5,Unovan,Crustle,Insectoid,Bug,Mineral,Bug,...,"[Fury Cutter, Sand Attack, Withdraw, Smack Dow...",50%:50%,70,105,125,65,75,45,"4,884-5,140","[Route 18, Mirage Spots, Ultra Space Wilds, Ax..."
3,559,1,5591,5,Unovan,Scraggy,Bipedal & Tailed,Dragon,Field,Dark,...,"[Low Kick, Leer, Payback, Headbutt, Sand Attac...",50%:50%,50,75,70,35,70,48,"3,599-3,855","[Route 1, Route 4, Route 18, P2 Laboratory, De..."
4,560,1,5601,5,Unovan,Scrafty,Bipedal & Tailed,Dragon,Field,Dark,...,"[Low Kick, Leer, Payback, Headbutt, Sand Attac...",50%:50%,65,90,115,45,115,58,"3,599-3,855","[Route 1, Route 15, Route 18, P2 Laboratory]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,904,1,9041,8,Hisuian,Overqwil,Finned,Unknown,Unknown,Dark,...,"[Poison Sting, Spikes, Pin Missile, Barb Barra...",Unknown,85,115,95,65,65,85,Unknown,[Unknown]
537,905,1,9051,8,Hisuian,Incarnate Enamorus,Head & Arms,Unknown,Unknown,Fairy,...,"[Tackle, Bite, Twister, Draining Kiss, Iron De...",Unknown,74,115,70,135,80,106,Unknown,[Unknown]
538,905,2,9052,8,Hisuian,Therian Enamorus,Quadrapedal,Unknown,Unknown,Fairy,...,"[Tackle, Bite, Twister, Draining Kiss, Iron De...",Unknown,74,115,110,135,100,46,Unknown,[Unknown]
539,905,1,9051,8,Hisuian,Incarnate Enamorus,Head & Arms,Unknown,Unknown,Fairy,...,"[Tackle, Bite, Twister, Draining Kiss, Iron De...",Unknown,74,115,70,135,80,106,Unknown,[Unknown]


<class 'pandas.core.frame.DataFrame'>
Final CSV Saved as: FinalDatabase2022.03.31-22.14.11.csv
Preserving: .\FinalDatabase2022.03.31-22.14.11.csv
Removing: .\temp2022.03.31-22.14.11.csv


### Section 6) Single Search - Research Display

In [46]:
#currentEntries, currentIndicies, NAME, DEXNUMB = pull_entries(fullListDF,'4') # Good Mega-Evo Example
#currentEntries, currentIndicies, NAME, DEXNUMB = pull_entries(fullListDF,'Yungoos') # Good Not-in-Gen-8 Example
#currentEntries, currentIndicies, NAME, DEXNUMB = pull_entries(fullListDF,'Meowth', '52') # Good Alt-Forms & Alt-Evolution Example
#currentEntries, currentIndicies, NAME, DEXNUMB = pull_entries(fullListDF,'26','Raichu') #

#SPECIAL_NAME_NUMBERS_LIST = [29, 32, 83, 122, 439, 678, 772, 865, 866, 876]

#ERROR AT 656 Darmanitan

currentEntries, currentIndicies, refName, NAME, DEXNUMB = pull_entries(fullListDF,'Meowth') # Good Not-in-Gen-8 Example

display(currentEntries)
print(currentIndicies)

Input 1 was Pokemon Name


,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
66,52,1,521,1,Kantonian,Meowth,,,,Normal,...,,,40,45,35,40,40,90,,
67,52,2,522,7,Alolan,Meowth,,,,Dark,...,,,40,35,35,50,40,90,,
68,52,3,523,8,Galarian,Meowth,,,,Steel,...,,,50,65,55,40,40,40,,


CURRENT INDICIES
[66, 67, 68]
Returning Entries for: #52 - Meowth


,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
66,52,1,521,1,Kantonian,Meowth,,,,Normal,...,,,40,45,35,40,40,90,,
67,52,2,522,7,Alolan,Meowth,,,,Dark,...,,,40,35,35,50,40,90,,
68,52,3,523,8,Galarian,Meowth,,,,Steel,...,,,50,65,55,40,40,40,,


[66, 67, 68]


In [47]:
print(refName)

BPimageDictionary = bulbapedia_images(NAME)

PDBtablesDictionary, PDBtableCount = pokemondb_tables(NAME)

SBtablesDictionary = serebii_tables(NAME, DEXNUMB)

Meowth
https://bulbapedia.bulbagarden.net/wiki/Meowth_(Pok%C3%A9mon)

Acquiring 685 Images for Meowth from Bulbapedia
[█████████████████████████] 100.0% ...Parsing

https://pokemondb.net/pokedex/Meowth

Acquiring 41 Tables for Meowth from PokemonDB
[█████████████████████████] 100.0% ...Collected

https://www.serebii.net/pokedex-swsh/meowth

Acquiring 113 Tables for Meowth from Serebii
[█████████████████████████] 100.0% ...Collected



In [48]:
print(refName)

BPparsed = bulbapedia_parser(BPimageDictionary, currentEntries, currentIndicies, refName)

PDBparsed = pokemondb_parser(PDBtablesDictionary, PDBtableCount, currentEntries, currentIndicies, NAME, refName)

SBparsed = serebii_parser(SBtablesDictionary, currentEntries, currentIndicies, refName)

alt_form_updater(NAME,currentIndicies, currentEntries)

alt_body_updater(NAME, currentIndicies, currentEntries)

display(currentEntries)

Meowth
66    \nUpdated Body Type for all entries of Meowth
67    \nUpdated Body Type for all entries of Meowth
68    \nUpdated Body Type for all entries of Meowth
Name: NAME, dtype: object
Meowth
Updated Egg Groups, Gender Ratios, Steps-To-Hatch, & Locations for all entries of Meowth
Standard Level Up added to Full Moves List
Sword & Shield Technical Machine Attacks added to Full Moves List
Technical Record Attacks added to Full Moves List
BDSP Technical Machine Attacks added to Full Moves List
Egg Moves (Details) added to Full Moves List
Usable Max Moves added to Full Moves List
FULL MOVE SET COMPILED
Alola Form Level Up added to Full Moves List
Sword & Shield Technical Machine Attacks added to Full Moves List
Technical Record Attacks added to Full Moves List
BDSP Technical Machine Attacks added to Full Moves List
Egg Moves (Details) added to Full Moves List
Usable Max Moves added to Full Moves List
FULL MOVE SET COMPILED
Galarian Form Level Up added to Full Moves List
Sword & Shield 

,NUMBER,CODE,SERIAL,GENERATION,REGION,NAME,BODY_TYPE,EGG_GROUP1,EGG_GROUP2,TYPE1,...,MOVES,GENDER_RATIOS(M:F),HP,ATK,DEF,SP_ATK,SP_DEF,SPD,STEPS_TO_HATCH,LOCATIONS
66,52,1,521,1,Kantonian,Meowth,Quadrapedal,Field,-,Normal,...,"[Fake Out, Growl, Feint, Scratch, Pay Day, Bit...",50%:50%,40,45,35,40,40,90,"4,884-5,140","[Route 5, Route 6, Route 7, Route 8, Route 5, ..."
67,52,2,522,7,Alolan,Meowth,Quadrapedal,Field,-,Dark,...,"[Fake Out, Growl, Feint, Scratch, Pay Day, Bit...",50%:50%,40,35,35,50,40,90,"4,884-5,140","[Route 5, Route 6, Route 7, Route 8, Route 5, ..."
68,52,3,523,8,Galarian,Meowth,Quadrapedal,Field,-,Steel,...,"[Fake Out, Growl, Hone Claws, Scratch, Pay Day...",50%:50%,50,65,55,40,40,40,"4,884-5,140","[Route 5, Route 6, Route 7, Route 8, Route 5, ..."
